## Hidden Markov Model (HMM)


A Markov chain is useful when we need to compute a probability for a sequence
of observable events. In many cases, however, the events we are interested in are
hidden: we don’t observe them directly. For example we don’t normally observe
part-of-speech tags in a text. Rather, we see words, and must infer the tags from the
word sequence.

In HMM the sequences are hidden because it is not possible to tell the state merely by the output symbol.

### Dynamic Programming (DP) Table
A dynamic programming table is a way of making your algorithm more efficient by storing some of the intermediate results. It's useful when your algorithm has a lot of repetitive computations. 

In [1]:
def dptable(state_prob):
    print(" ".join(("%8d" % i) for i in range(state_prob.shape[0])))
    for i, prob in enumerate(state_prob.T):
        print("%.7s: " % states[i] +" ".join("%.7s" % ("%f" % p) for p in prob))

Let's consider a simple HMM with:
* 2 hidden states: Healthy or Fever
* 3 possible observations: Normal, Cold or Dizzy

The model can then be used to predict if a person is feverish, at every time-step, from a given observation sequence. There are several paths through the hidden states (Healthy and Fever) that lead to the given sequence, each with different paths.

In [ ]:
import pandas as pd
import numpy as np

### Model Parameters 

* **Transition probability**: 
    * A matrix with the probabilities from transitioning from one state to the next state, over time.
    * P(nextState|currentState)


* **Emission probability**: 
    * A matrix with the probabilities of an observation (output) being generated from a state.
    * P(Observation|currentState)


* **Initial probability**: 
    * A starting probability distribution over states
    * P(initialState)


* **Final probability**: A final probability distribution over states

In [ ]:
# A start distribution π is required; setting π = [0.6, 0.4] means a probability 0.6 of starting in state 1 (Healthy), 
# and a  probability 0.4 of starting in state 2 (Fever)
p0 = np.array([0.6, 0.4])

transition = np.array([[0.7, 0.3],
                       [0.4, 0.6]])

emissions = np.array([[0.5, 0.1], 
              [0.4, 0.3], 
              [0.1, 0.6]])

states = {0:'Healthy', 1:'Fever'}
observations = {0:'normal', 1:'cold', 2:'dizzy'}

obs_seq = np.array([0, 0, 1, 2, 2]) # Sequence: Normal, Normal, Cold, Dizzy, Dizzy

## Initial state vector
We can think of the _i_th component of the vector as representing the probability that the chain starts in state $s_{i}$ 

In [5]:
df_p0 = pd.DataFrame(p0, index=["Healthy", "Fever"], columns=["Probability"])
df_p0

,Probability
Healthy,0.6
Fever,0.4


## Transition probability matrix
Let's assume that if the person is Healthy now the probability that they will be healthy the next _"time"_ is 70%. So that leaves a 30% chance that the person will be Feverish the next _"time"_. 

Let's assume that if the person is Feverish now, there's a 40% chance that they'll become Healthy next _"time"_ and a 60% chance that they'll remain feverish next _"time"_.

In [ ]:
df_transition = pd.DataFrame(transition, index=["fromHealthy", "fromFever"], columns=["toHealthy", "toFever"])
df_transition

,toHealthy,toFever
fromHealthy,0.7,0.3
fromFever,0.4,0.6


## Emission probability marix (Observation)
Let's say that we have concluded that when a person is Healthy there's a 50% chance that they are Normal, a 40% chance that they are Cold, and a 10% chance that they are Dizzy.

Let's say that we also concluded that when a person is Feverish there's a small 10% chance they are Normal, a 30% chance that they are Cold, and a much larger 60% chance that they are Dizzy.

In [ ]:
df_emissions = pd.DataFrame(emissions, index=["Normal", "Cold", "Dizzy"], columns=["Healthy", "Fever"])
df_emissions

,Healthy,Fever
Normal,0.5,0.1
Cold,0.4,0.3
Dizzy,0.1,0.6


## Run Viterbi
Calculate the Viterbi path, which is the most likely sequence of states that generated the sequence given the full model

In [8]:
from HiddenMarkovModel import HiddenMarkovModel # Don't forget to add the local file HiddenMarkovModel.py to your present working directory

In [9]:
model = HiddenMarkovModel(transition, emissions, p0) # Define the HMM

states_seq, state_prob = model.viterbi_inference(obs_seq) # Calculate the Viterbi path

In [10]:
print("Observation sequences: ", [observations[o] for o in obs_seq])

Observation sequences:  ['normal', 'normal', 'cold', 'dizzy', 'dizzy']


In [11]:
import torch

# Transpose the state probability matrix. Move the tensor to cpu(). Convert torch tensor to numpy array. 
df = pd.DataFrame(torch.t(state_prob).cpu().numpy(), index=["Health", "Fever"])

def highlight_max(s):
    '''
    Highlight the maximum in a series in yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

# Display the tensor
df.style.apply(highlight_max, axis=0)

,0,1,2,3,4
Health,0.3,0.105,0.0294,0.002058,0.00021168
Fever,0.04,0.009,0.00945,0.005292,0.00190512


In [12]:
print("The most likely states are: ", [states[s.item()] for s in states_seq])

The most likely states are:  ['Healthy', 'Healthy', 'Healthy', 'Fever', 'Fever']
